In [1]:
import pandas as pd
import xlrd
#import arcpy
import openpyxl
from pandas import ExcelWriter
from datetime import datetime
import os
#from arcgis.gis import GIS
#from arcgis.features import SpatialDataFrame
from shutil import copyfile
from openpyxl.utils.cell import get_column_letter
import urllib.request, json
import new_geopoll_functions

In [2]:
###USER DEFINED INPUTS BEGINNING
template_version = "20230720"
iso3 = "PSE"
round_number = "1" #Specify the round number
admin_level = "Admin 2"
validation_number = 1 #For now this is not being used, keep it as 1 ; Please enter 1 or 2 ONLY
user_name = "Ibrahim Ali Ahmad" #Please write your name
country_questionnaire_file = r'household_questionnaire_geopoll_AR_20230720_WestBank_FAOPAL_DIEM review - For validation - NEW.xlsx' #the questionnaire file that we use for creating the table

previousround_questionnaire = "no" #yes if questionnaire built based on previous questionnaire validated; no if questionnaire built from scratch using template
previousround_questionnaire_file = r'.xlsx' #the previous validated questionnaire file that we use for creating the current one

## Modfiy path to respiratory
template_questionnaire_location = r'/Users/ibrahim.aliahmad/Documents/FAO/Working Files/Questionnaire Validation/Script/questionnaire_validation/Questionnaire Validation/Questionnaires/Questionnaire Templates'
previousround_questionnaire_location = r'/Users/ibrahim.aliahmad/Documents/FAO/Working Files/Questionnaire Validation/Script/questionnaire_validation/Questionnaire Validation/Questionnaires/Questionnaire Previous Round'
country_questionnaires_location = r'/Users/ibrahim.aliahmad/Documents/FAO/Working Files/Questionnaire Validation/Script/questionnaire_validation/Questionnaire Validation/Questionnaires/Questionnaire Country'
output_locations = r'/Users/ibrahim.aliahmad/Documents/FAO/Working Files/Questionnaire Validation/Script/questionnaire_validation/Questionnaire Validation/Questionnaires/Validated Questionnaires/PSE/R1/V3'
##

###USER DEFINED INPUTS END

In [3]:
enumerator = new_geopoll_functions.detect_enumerator(country_questionnaire_file)
lang = new_geopoll_functions.detect_language(country_questionnaire_file)
startTime = datetime.now()
now = datetime.now().strftime('%Y%m%d%H%M%S')

from datetime import datetime
now1 = datetime.now()
formatted_now = now1.strftime("%d/%m/%Y, %H:%M:%S")

execution_messages = ''
execution_messages += "\n%s%s" % ("Country iso3: ", iso3)
execution_messages += "\n%s%s" % ("Country round #: ", round_number)
execution_messages += "\n%s%s" % ("Country Questionnaire Language: ", lang)
execution_messages += "\n%s%s" % ("Country Questionnaire Type: ", enumerator)
execution_messages += "\n%s%s" % ("Country Questionnaire File: ", country_questionnaire_file)



country_questionnaire_file = os.path.join(country_questionnaires_location, country_questionnaire_file)
template_questionnaire_file = new_geopoll_functions.detect_template(template_version,country_questionnaire_file)
template_questionnaire_file = os.path.join(template_questionnaire_location, template_questionnaire_file)
execution_messages += "\n%s%s" % ("The template used to validate the country questionnaire is: ", template_version)   

if previousround_questionnaire == "yes":
    execution_messages += "\n%s%s" % ("Previous Questionnaire used: ", previousround_questionnaire_file)   
    previousround_questionnaire_file = os.path.join(previousround_questionnaire_location, previousround_questionnaire_file)

execution_messages += "\n%s%s" % ("Validation number: ", validation_number)
execution_messages += "\n%s%s" % ("User: ", user_name)
execution_messages += "\n%s%s" % ("Validated on: ", formatted_now)

print(execution_messages)






Country iso3: PSE
Country round #: 1
Country Questionnaire Language: ar
Country Questionnaire Type: geopoll
Country Questionnaire File: household_questionnaire_geopoll_AR_20230720_WestBank_FAOPAL_DIEM review - For validation - NEW.xlsx
The template used to validate the country questionnaire is: 20230720
Validation number: 1
User: Ibrahim Ali Ahmad
Validated on: 09/04/2024, 21:51:28


In [4]:
##CHECK THE QUESTIONNAIRE BEFORE PROCEEDING TO THE NEXT CELL##
print("Checking the conditionality of all questions")


questions_conditionality = os.path.join(output_locations, "conditionality_questionnaire_%s_%s_%s_%s.xlsx" % (enumerator,lang,iso3, now)) 


conditionality_result_brief, conditionality_result_details = new_geopoll_functions.check_all_questions(country_questionnaire_file, template_questionnaire_file, questions_conditionality)

print(conditionality_result_brief)
execution_messages += "\n\n" + conditionality_result_details

Checking the conditionality of all questions

detecting differences in column: Suggested Name

detecting differences in column: English

detecting differences in column: Length

detecting differences in column: Q Type

detecting differences in column: Conditional

detecting differences in column: Skip Pattern

detecting differences in column: Codes

detecting differences in column: Mandatory



In [5]:
print("Reading TEMPLATE questionnaire and creating recap excel file")
questionnaire_file = template_questionnaire_file
temp_path = output_locations

coded_values_file = os.path.join(temp_path, "coded_values_%s_template.xlsx" % now) #intermediary output file with all categories and codes extracted from the questionnaire
field_names_list = []
max_counter = 6000 #for testing purposes, we may need to limit the execution only to some items
languages = new_geopoll_functions.read_questionnaire(questionnaire_file, coded_values_file)

print("Reading COUNTRY questionnaire and creating recap excel file")
country_coded_values_file = os.path.join(temp_path, "coded_values_%s_country.xlsx" % now) #intermediary output file with all categories and codes extracted from the questionnaire
field_names_list = []
max_counter = 6000 #for testing purposes, we may need to limit the execution only to some items
languages = new_geopoll_functions.read_questionnaire(country_questionnaire_file, country_coded_values_file)


Reading TEMPLATE questionnaire and creating recap excel file
Version in Arabic and English
Reading COUNTRY questionnaire and creating recap excel file
Version in Arabic and English
Failed question o_crp_proddif_conflict_damage

1) o_crp_pd_confl_dam_confiscation
2) o_crp_pd_confl_dam_water_infra
3) o_crp_pd_confl_dam_agri_prod
4) o_crp_pd_confl_dam_farm_infra
5) o_crp_pd_confl_dam_dk
6) o_crp_pd_confl_dam_ref (type: Single Choice) for error: Excel worksheet name 'o_crp_proddif_conflict_damage

1) o_crp_pd_confl_dam_confiscation
2) o_crp_pd_confl_dam_water_infra
3) o_crp_pd_confl_dam_agri_prod
4) o_crp_pd_confl_dam_farm_infra
5) o_crp_pd_confl_dam_dk
6) o_crp_pd_confl_dam_ref' must be <= 31 chars. 
Failed question o_crp_proddif_conflict_damage

1) o_crp_pd_confl_dam_confiscation
2) o_crp_pd_confl_dam_water_infra
3) o_crp_pd_confl_dam_agri_prod
4) o_crp_pd_confl_dam_farm_infra
5) o_crp_pd_confl_dam_dk
6) o_crp_pd_confl_dam_ref (type: Single Choice) for error: Excel worksheet name 'o_crp_

In [6]:
print ("COUNTING NUMBER OF QUESTIONS")

T_sqname_list_questions, T_sqname_n_of_questions = new_geopoll_functions.count_number_of_questions_sqname(questionnaire_file)
T_qname_list_questions, T_qname_n_of_questions = new_geopoll_functions.count_number_of_questions_qname(questionnaire_file)
C_sqname_list_questions, C_sqname_n_of_questions = new_geopoll_functions.count_number_of_questions_sqname(country_questionnaire_file)
C_qname_list_questions, C_qname_n_of_questions = new_geopoll_functions.count_number_of_questions_qname(country_questionnaire_file)

message1 = "\nQName Field - Number of questions in the Template Questionnaire: %s" % T_qname_n_of_questions
message2 = "QName Field - Number of questions in the Country Questionnaire: %s" % C_qname_n_of_questions

if previousround_questionnaire == "yes":
    P_sqname_list_questions, P_sqname_n_of_questions = new_geopoll_functions.count_number_of_questions_sqname(previousround_questionnaire_file)
    P_qname_list_questions, P_qname_n_of_questions = new_geopoll_functions.count_number_of_questions_qname(previousround_questionnaire_file)
    message2P = "QName Field - Number of questions in the Previous Questionnaire: %s\n" % P_qname_n_of_questions

message3 = "\nSuggestedQName Field - Number of questions in the Template Questionnaire: %s" % T_sqname_n_of_questions
message4 = "SuggestedQName Field - Number of questions in the Country Questionnaire: %s" % C_sqname_n_of_questions

if previousround_questionnaire == "yes":
    message4P = "SuggestedQName Field - Number of questions in the Country Questionnaire: %s" % P_sqname_n_of_questions

if previousround_questionnaire == "yes":
    execution_messages += "\n%s\n%s\n%s\n%s\n%s\n%s" % (message1, message2, message2P, message3, message4, message4P)
    print(message1)
    print(message2)
    print(message2P)
    print(message3)
    print(message4)
    print(message4P)

    
else:
    execution_messages += "\n%s\n%s\n%s\n%s" % (message1, message2, message3, message4)
    print(message1)
    print(message2)
    print(message3)
    print(message4)




COUNTING NUMBER OF QUESTIONS

QName Field - Number of questions in the Template Questionnaire: 181
QName Field - Number of questions in the Country Questionnaire: 134

SuggestedQName Field - Number of questions in the Template Questionnaire: 179
SuggestedQName Field - Number of questions in the Country Questionnaire: 132


In [7]:
print ("CHECKING THAT ALL MANDATORY QUESTIONS ARE INCLUDED IN THE COUNTRY QUESTIONNAIRE")

# Load the questionnaire file from an excel file into a pandas DataFrame; we skip the first 2 rows as they are headers or metadata
quest_df = pd.read_excel(open(questionnaire_file, 'rb'), sheet_name='survey',skiprows=2)

# From the DataFrame, get a list of mandatory question names that don't contain "hh_wealth_" in their name
# The list is derived from rows where "Q Name" doesn't contain "hh_wealth_" and "Mandatory" is set to "yes"
# Values of "Q Name" are converted to list and any leading/trailing whitespaces in each question name is removed
list_mandatory_questions = quest_df.loc[((quest_df["Q Name"].str.contains("hh_wealth_") == False ) & (quest_df["Mandatory"] == "yes"))]["Q Name"].values.tolist() 
list_mandatory_questions = [s.strip() for s in list_mandatory_questions]
message9 = "\nFound %s mandatory questions in the template questionnaire" % len(list_mandatory_questions)
print(message9)

# Check if all mandatory questions are present in the country questionnaire (C_qname_list_questions)
result =  all(elem in C_qname_list_questions  for elem in list_mandatory_questions)
if result:
    message10 = "Yes, country questionnaire contains all mandatory questions"   
else:
    message10 = "No, country questionnaire does not contains all mandatory questions. Missing question: %s " % list(set(list_mandatory_questions ) - set(C_qname_list_questions))
print(message10)
    
execution_messages += "\n%s\n%s" % (message9, message10)

CHECKING THAT ALL MANDATORY QUESTIONS ARE INCLUDED IN THE COUNTRY QUESTIONNAIRE

Found 81 mandatory questions in the template questionnaire
No, country questionnaire does not contains all mandatory questions. Missing question: ['income_sec_fish_check', 'need_received', 'fish_main', 'income_third_comp', 'income_third_fish_check', 'income_third_crop_check', 'fish_proddif', 'income_third_ls_check', 'fish_change', 'income_main_crop_check', 'income_sec_ls_check', 'income_main_fish_check', 'fish_salesdif_2', 'fish_intro', 'fish_salesdif', 'fish_proddif_2', 'income_third', 'fish_salesmain', 'resp_firstname', 'income_sec_comp', 'fish_salesprice', 'income_sec', 'resp_lastname', 'need_2', 'income_main_ls_check', 'income_sec_crop_check'] 


In [8]:
print ("DETECTING DIFFERENCES IN FIELD: Suggested QName")

unique_country_questionnaire = list(set(C_sqname_list_questions) - set(T_sqname_list_questions))
unique_template_questionnaire = list(set(T_sqname_list_questions) - set(C_sqname_list_questions))

message5 = "\nSuggestedQName Field - Questions in country questionnaire only (therefore not stored/published on the Hub):"
for question in unique_country_questionnaire:
    message5 += f"\n- {question}"

message6 = "\nSuggestedQName Field - Questions in template questionnaire only (therefore not collected in the country):"
for question in unique_template_questionnaire:
    message6 += f"\n- {question}"

execution_messages += "\n%s\n%s" % (message5, message6)

print (message5)
print (message6)

DETECTING DIFFERENCES IN FIELD: Suggested QName

SuggestedQName Field - Questions in country questionnaire only (therefore not stored/published on the Hub):
- o_income_tot_amount
- o_conflict_any
- o_income_other

1) o_income_ag_prod_staple_crp
2) o_income_ag_prod_veg_fruit
3) o_income_ag_prod_cash_crp
4) o_income_ag_prod_livestock
5) o_income_ag_prod_bee
6) o_income_ag_prod_aquatic
7) o_income_ag_prod_forestry
8) o_income_ag_trade_nonprod
9) o_income_ag_dailywage
10) o_income_ag_stable_employ
11) o_income_nonag_selfemploy
12) o_income_nonag_dailywage
13) o_income_nonag_stable_employ
14) o_income_nonag_public_employ
15) o_income_charity
16) o_income_welfare
17) o_income_remittances
18) o_income_rents
19) o_income_no_other
20) o_income_dk
21) o_income_ref
- o_ls_salesdif_conflict

1) o_ls_salesdif_conf_mkt_access
2) o_ls_salesdif_conf_transp
3) o_ls_salesdif_conf_loss_storage
4) o_ls_salesdif_conf_lack_demand
5) o_ls_salesdif_conf_pay_delay
6) o_ls_salesdif_conf_process
7) o_ls_salesdif

In [9]:
print ("DETECTING DIFFERENCES IN FIELD: Q Name")

unique_country_questionnaire_qname = list(set(C_qname_list_questions) - set(T_qname_list_questions))
unique_template_questionnaire_qname = list(set(T_qname_list_questions) - set(C_qname_list_questions))

message7 = "\nQName Field - Questions in country questionnaire only (therefore not stored/published on the Hub):"
for question in unique_country_questionnaire_qname:
    message7 += f"\n- {question}"

message8 = "\nQName Field - Questions in template questionnaire only (therefore not collected in the country):"
for question in unique_template_questionnaire_qname:
    message8 += f"\n- {question}"

execution_messages += "\n%s\n%s" % (message7, message8)

print (message7)
print (message8)

DETECTING DIFFERENCES IN FIELD: Q Name

QName Field - Questions in country questionnaire only (therefore not stored/published on the Hub):
- o_income_tot_amount
- o_conflict_any
- o_income_other
- o_crp_proddif_conflict
- o_hh_wealth_electricity
- o_income_before
- o_ls_proddif_conflict_damage
- o_conflict_agri
- o_assistance_provider
- o_ls_proddif_settler
- o_income_before_change
- o_movement_any
- o_income_tot_amount_conf
- o_displace_where
- o_need_2
- o_conflict_type
- o_hh_area
- o_income_before_mitigate
- o_hh_wealth_water
- o_displace_any
- o_need_received
- o_hh_wealth_toilet
- o_crp_sales_markets
- o_income_before_ag
- o_crp_proddif_conflict_damage
- o_income_before_lost
- o_ls_proddif_conflict
- o_movement_type
- o_hh_wealth_water_drinking
- o_conflict_food
- o_crp_salesdif_conflict
- o_ls_salesdif_conflict

QName Field - Questions in template questionnaire only (therefore not collected in the country):
- cs_crisis_harv_immature_crops
- fcs_meat_fish_days
- fish_main
- incom

In [10]:
print ("CHECKING THAT RULES ON HH_WEALTH_ QUESTIONS ARE RESPECTED\n")

wealth_questions_in_country = [i for i in C_qname_list_questions if i.startswith('hh_wealth_')]
if len(wealth_questions_in_country) >= 1:
    message11 = "\n1 hh_wealth_ question found in the country questionnaire: approved\n"
else:
    message11 = message11 = "\n%s hh_wealth_ questions found in the country questionnaire: it should be at least one!\n" % len(wealth_questions_in_country)

print(message11)

execution_messages += "\n%s" % (message11)

CHECKING THAT RULES ON HH_WEALTH_ QUESTIONS ARE RESPECTED


0 hh_wealth_ questions found in the country questionnaire: it should be at least one!



In [11]:
print ("CHECKING THAT RULES ON CS QUESTIONS ARE RESPECTED\n")

cs_questions_in_country = [i for i in C_qname_list_questions if i.startswith('cs')]
if len(cs_questions_in_country) > 0:
    cs_stress_questions_in_country = [i for i in C_qname_list_questions if i.startswith('cs_stress')];
    cs_emergency_questions_in_country = [i for i in C_qname_list_questions if i.startswith('cs_emergency')];
    cs_crisis_questions_in_country = [i for i in C_qname_list_questions if i.startswith('cs_crisis')];
    
    if (len(cs_stress_questions_in_country) >= 4) & (len(cs_emergency_questions_in_country)>=3) & (len(cs_crisis_questions_in_country) >= 3):
        message111 = "\n%s CS questions found in the country questionnaire\n" % len(cs_questions_in_country) + "There are more than 10 coping strategies included in the questionnaire. Make sure that each respondent profile will be administered at least 10 coping strategies.\nNumber of STRESS questions: %s" % len(cs_stress_questions_in_country) + "\nNumber of CRISIS questions: %s\n" % len(cs_crisis_questions_in_country) + "Number of EMERGENCY questions: %s\n" % len(cs_emergency_questions_in_country) + "Rule respected"
        
    else: 
        message111 = "\n%s CS questions found in the country questionnaire\n" % len(cs_questions_in_country) + "There are less than 10 coping strategies included in the questionnaire. Add coping strategies so that each respondent profile receives at least 10 coping strategies.\nNumber of STRESS questions: %s" % len(cs_stress_questions_in_country) + "\nNumber of CRISIS questions: %s\n" % len(cs_crisis_questions_in_country) + "Number of EMERGENCY questions: %s\n" % len(cs_emergency_questions_in_country) + "Rule NOT respected"

else:
    message111 = "\nNo CS questions in the country questionnaire.\nThere are less than 10 coping strategies included in the questionnaire. Add coping strategies so that each respondent profile receives at least 10 coping strategies."

print(message111)

execution_messages += "\n%s" % (message111)

CHECKING THAT RULES ON CS QUESTIONS ARE RESPECTED


11 CS questions found in the country questionnaire
There are more than 10 coping strategies included in the questionnaire. Make sure that each respondent profile will be administered at least 10 coping strategies.
Number of STRESS questions: 4
Number of CRISIS questions: 4
Number of EMERGENCY questions: 3
Rule respected


In [12]:
print ("CHECKING THAT RULES ON HDDS QUESTIONS ARE RESPECTED")

hdds_questions_in_country = [i for i in C_qname_list_questions if i.startswith('hdds')]
message12 = "\nThere should be more than 2 HDDS fields"

if len(hdds_questions_in_country) > 0:
    message12 += "\nNumber of HDDS questions: %s  - %s" % (len(hdds_questions_in_country), hdds_questions_in_country)
else:
    message12 += "\nNo HDDS questions in the country questionnaire"

execution_messages += "\n%s" % (message12)

print(message12)

CHECKING THAT RULES ON HDDS QUESTIONS ARE RESPECTED

There should be more than 2 HDDS fields
Number of HDDS questions: 2  - ['hdds', 'hdds_confirmation']


In [13]:
print ("CHECKING THAT RULES ON CROP HARVEST QUESTIONS ARE RESPECTED")

crp_questions_in_country = [i for i in C_qname_list_questions if (i.startswith('crp_harv') and i != ('crp_harv_lastyr'))]

message09 = "\nThere should be either only \"crp_harv_change\" or the 4 crp_harv questions from the template: "

if (len(crp_questions_in_country) == 4) and ("crp_harv_change" in crp_questions_in_country):
    message09 += "RULE IS RESPECTED\nNumber of crp_harv questions: %s  - %s" % (len(crp_questions_in_country), crp_questions_in_country)
    
elif (len(crp_questions_in_country) == 1) and ("crp_harv_change" in crp_questions_in_country):
    message09 += "RULE IS RESPECTED\nNumber of crp_harv questions: %s  - %s" % (len(crp_questions_in_country), crp_questions_in_country)
else:
    message09 += "RULE IS NOT RESPECTED\nNumber of crp_harv questions: %s  - %s" % (len(crp_questions_in_country), crp_questions_in_country)

execution_messages += "\n%s" % (message09)

print(message09)

CHECKING THAT RULES ON CROP HARVEST QUESTIONS ARE RESPECTED

There should be either only "crp_harv_change" or the 4 crp_harv questions from the template: RULE IS RESPECTED
Number of crp_harv questions: 4  - ['crp_harv_change', 'crp_harv_vol', 'crp_harv_unit', 'crp_harv_unit_kg']


In [14]:
import pandas as pd
 
print("Comparison of Domains - Marked Change from Template\n")
 
# Function to compare domains between template and country for a specific column, with enhanced output formatting
def compare_and_print_differences(template_df, country_df, column_index):
    for index, template_row in template_df.iterrows():
        q_name = template_row.iloc[1]  # Access QName from the second column (index 1)
 
        # Proceed only if QNAME exists in the country dataframe
        country_row = country_df[country_df.iloc[:, 1] == q_name]
        if not country_row.empty:
            template_domain_text = str(template_row.iloc[column_index]).split('\n')
            country_domain_text = str(country_row.iloc[0, column_index]).split('\n')
           
            # Print differences, now omitting QNames not found and formatting output
            differences_found = False
            for i, template_item in enumerate(template_domain_text, start=1):
                country_item = country_domain_text[i - 1] if i <= len(country_domain_text) else 'MISSING'
                if template_item != country_item:
                    if not differences_found:
                        print(f"\nQName: {q_name}")
                        differences_found = True
                    #print(f"Option {i} Difference:")  remove!!!
                    print(f"Template -> {template_item}")
                    print(f"Country -> {country_item}")
            if differences_found:
                print("\n-----\n")

                
template_df = pd.read_excel(template_questionnaire_file, header=None, skiprows=2)
country_df = pd.read_excel(country_questionnaire_file, header=None, skiprows=2)
 
# English descriptions comparison (Column D is at index 3)
compare_and_print_differences(template_df, country_df, 3)
 
# Other language descriptions comparison (Column F is at index 5)
compare_and_print_differences(template_df, country_df, 5)
 
print("Comparison of Domains - Marked Change from Template - ENTIRE OUTPUT\n")
 
# Load the template and country questionnaire files into pandas DataFrames
template_df = pd.read_excel(template_questionnaire_file, header=2)  # Skip the first two rows
country_df = pd.read_excel(country_questionnaire_file, header=2)  # Skip the first two rows
 
# Keep track of whether any changes are found
changes_found = False
 

# Iterate over columns B and D
for template_qname, template_english in zip(template_df.iloc[:, 1], template_df.iloc[:, 3]):
    # Iterate over rows of the country questionnaire to find corresponding values
    found_match = False
    for index, country_row in country_df.iterrows():
        if country_row.iloc[1] == template_qname:
            country_english = country_row.iloc[3]
            if country_english != template_english:
                print(f"QName: {template_qname}")
                print(f"Template English: {template_english}")
                print(f"\nCountry English: {country_english}\n")
                changes_found = True
            found_match = True
            break  # Stop searching for the template value in the country questionnaire once found
    if not found_match:
        changes_found = True  # If no match is found, consider it as a change

# If no changes are found in column D, print a message
if not changes_found:
    print("No differences found in column D between the template and the country questionnaire.")

# Keep track of whether any changes are found in column F
changes_found_column_f = False
 
# Iterate over columns B and F
for template_qname, template_value_f in zip(template_df.iloc[:, 1], template_df.iloc[:, 5]):
    # Iterate over rows of the country questionnaire to find corresponding values
    found_match = False
    for index, country_row in country_df.iterrows():
        if country_row.iloc[1] == template_qname:
            country_value_f = country_row.iloc[5]
            if country_value_f != template_value_f:
                print(f"QName: {template_qname}")
                print(f"Template Value F: {template_value_f}")
                print(f"\nCountry Value F: {country_value_f}\n")
                changes_found_column_f = True
            found_match = True
            break  # Stop searching for the template value in the country questionnaire once found
    if not found_match:
        changes_found_column_f = True  # If no match is found, consider it as a change

# If no changes are found in column F, print a message
if not changes_found_column_f:
    print("No differences found in column F between the template and the country questionnaire.")

Comparison of Domains - Marked Change from Template


QName: introduction
Template -> You will receive #TOPUP#! of communication credit as an incentive for the participation of the survey.
Country -> 
Template -> 
Country -> Please note that this call may be recorded for quality purposes.
Template -> Please note that this call may be recorded for quality purposes.
Country -> 
Template -> 
Country -> Do you have any questions?
Template -> Do you have any questions?
Country -> 
Template -> 
Country -> [OPERATOR: ADDRESS QUESTIONS FROM RESPONDENT, IF ANY.]
Template -> [OPERATOR: ADDRESS QUESTIONS FROM RESPONDENT, IF ANY.]
Country -> 
Template -> 
Country -> 1)CONTINUE
Template -> 1)CONTINUE
Country -> MISSING

-----


QName: crp_storage
Template -> Last year, what proportion of your {crp_main} stock did you lose [to pests, rotting, mould, etc.] after the harvest?
Country -> Last year, what proportion of your {crp_main} stock did you lose [to pests, rotting, mould, security situation etc.]

QName: crp_storage
Template English: Last year, what proportion of your {crp_main} stock did you lose [to pests, rotting, mould, etc.] after the harvest?

[OPERATOR: SINGLE RESPONSE.]

1)None
2)A little
3)An important part of it, but less than half
4)Most of it [half or more]
5)Almost everything 
6)Everything
7)DON'T KNOW
8)REFUSED

Country English: Last year, what proportion of your {crp_main} stock did you lose [to pests, rotting, mould, security situation etc.] after the harvest?

[OPERATOR: SINGLE RESPONSE.]

1)None
2)A little
3)An important part of it, but less than half
4)Most of it [half or more]
5)Almost everything 
6)Everything
7)DON'T KNOW
8)REFUSED

QName: ls_main
Template English: What was the main animal raised by your household for food or income generation in the last 12 months?

[OPERATOR: SINGLE RESPONSE. ASK ABOUT THE MOST IMPORTANT ANIMAL ONLY.] [HINT: DON'T FORGET TO PROBE ABOUT TRANSHUMANCE - MOVING LIVESTOCK FROM ONE GRAZING GROUND TO ANOTHER.]

1)Cattle [cow / be

QName: ls_main
Template Value F: ما هو الحيوان الرئيسي الذي قامت أسرتك بتربيته للاستخدام الغذائي أو لتوليد الدخل خلال الاثني عشر شهرًا الماضية؟

[المشغل: استجابة واحدة. اسأل عن الحيوان الأكثر أهمية فقط.] [ملحوظة: لا تنسَ التحقيق في مسألة التنقل الرعوي - نقل االماشية من أرض رعي إلى أخرى.]

1) البقر [بقرة / ثور / عجل  / ياك / جاموس]
2) الماعز
3) الأغنام
4) الخنازير [خنزير / خنزير بري / إلخ]
5) الحمير/ الأحصنة
6) الثدييات المستأنسة الصغيرة [الأرانب / المنك / خنازير غينيا / إلخ]
7) دواجن [دجاج / دجاج غينيا / بطة / سمان / ديك رومي / إلخ]
8) الإبل
9) آخر
10) لا أعرف
11) رفض الإجابة

Country Value F: ما هو الحيوان الرئيسي الذي قامت أسرتك بتربيته للاستخدام الغذائي أو لتوليد الدخل خلال الاثني عشر شهرًا الماضية؟

[المشغل: استجابة واحدة. اسأل عن الحيوان الأكثر أهمية فقط.] [ملحوظة: لا تنسَ التحقيق في مسألة التنقل الرعوي - نقل االماشية من أرض رعي إلى أخرى.]

1) البقر [بقرة / ثور / عجل  / ياك / جاموس]
2) الماعز
3) الأغنام
4) الحمير/ الأحصنة
5) الثدييات المستأنسة الصغيرة [الأرانب / المنك / خنازير غين

In [15]:
print ("CREATE OUTPUT VALIDATED QUESTIONNAIRE FILE")
country_questionnaire_edited_file = os.path.join(temp_path, "validated_questionnaire_%s_%s_%s_%s.xlsx" % (enumerator,lang,iso3, now)) 
copyfile(country_questionnaire_file, country_questionnaire_edited_file)
message =  "\n\nOutput validated country questionnaire: %s" % (country_questionnaire_edited_file)
print(message)
execution_messages += message

CREATE OUTPUT VALIDATED QUESTIONNAIRE FILE


Output validated country questionnaire: /Users/ibrahim.aliahmad/Documents/FAO/Working Files/Questionnaire Validation/Script/questionnaire_validation/Questionnaire Validation/Questionnaires/Validated Questionnaires/PSE/R1/V3/validated_questionnaire_geopoll_ar_PSE_20240409215128.xlsx


In [16]:
print("INSERT ADMIN SHEET WITH UPDATED BOUNDARIES")
df = new_geopoll_functions.insert_sheet_with_adm_reference(country_questionnaire_edited_file,admin_level,iso3)
df.head()
execution_messages += "\n\nCreated Admin Info sheet with updated boundaries."

INSERT ADMIN SHEET WITH UPDATED BOUNDARIES


In [17]:
print("EDITING THE SURVEY BY SORTING THE CROP LIST VALUES ACCORDING TO PRIORITISE")
n_of_choices = new_geopoll_functions.sort_crop_list_by_selection(country_questionnaire_edited_file)
message13 = "\nNumber of most common crops selected in the Crop list sheet: %s" % n_of_choices
print(message13)
execution_messages += "\n\nSurvey edited by sorting the crop list according to the prioritised crops" + message13

EDITING THE SURVEY BY SORTING THE CROP LIST VALUES ACCORDING TO PRIORITISE
0      1
1      2
2      3
3      4
4      5
      ..
85    86
86    87
87    88
88    89
89    90
Name: GeoPoll code, Length: 90, dtype: int64

Number of most common crops selected in the Crop list sheet: 21


In [18]:
##CHECK THE QUESTIONNAIRE BEFORE PROCEEDING TO THE NEXT CELL##
##IF ALL CELLS ARE FILLED ON THE QUESTIONNAIRE TO BE VALIDATED##

print("EDITING THE SURVEY BY REPLACING VALUES ACCORDING ADDITIONAL INFO SHEET")

if previousround_questionnaire == "yes":
    new_geopoll_functions.update_questionnaire(template_questionnaire_file,country_questionnaire_edited_file)

new_geopoll_functions.find_and_replace_strings_in_df(country_questionnaire_edited_file)


execution_messages += "\n\nSurvey edited by replacing values according the Additional Information sheet"

EDITING THE SURVEY BY REPLACING VALUES ACCORDING ADDITIONAL INFO SHEET


In [19]:
print("Create output file with execution messages")
text_file = os.path.join(temp_path, "questionnaire_validation_report_%s_R%s_%s.txt" % (iso3,round_number,now)) 
file = open(text_file, "w") 
file.write(execution_messages) 
file.close() 

Create output file with execution messages


In [20]:
print("Execution time: ", datetime.now() - startTime)

Execution time:  0:03:22.622715
